In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time
import datetime as dt
import re

from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets

from matplotlib import pyplot as plt
import seaborn as sns

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:79% !important; }</style>"))

# easier-to-read notebook:
from IPython.display import display, HTML, clear_output
pd.options.display.max_columns = 70
pd.options.display.max_rows = 500
%load_ext autoreload
%autoreload 2

r = re.compile('.*maoz*.')
if len(list(filter(r.match, sys.path))) != 0:
    PHYTECH_DRIVE_PATH = os.environ['PHYTECH_DRIVE_PATH']
    #PHYTECH_DRIVE_PATH = '/Users/maozdotan/Google Drive/My Drive'
    GITHUB_PATH = PHYTECH_DRIVE_PATH + '/GitHub'
    DATA_WD = PHYTECH_DRIVE_PATH + '/Data Integrity/SM anomalies/data'
    CERT_PATH = PHYTECH_DRIVE_PATH + '/Data'
    if CERT_PATH not in sys.path:
        sys.path.append(CERT_PATH)
    from sql_import_export import SqlImporter
    import cert_aws as c
    sql_importer = SqlImporter(database = c.database_research, user = c.user_research, password = c.password_research,
                                host = c.host_research, port = c.port_research, verbose=True)

local aws_cert.py loaded


In [84]:
%reload_ext autoreload
import project_class_data_extract
import logic_parameters# import default_latitude, default_height
from tqdm import tqdm_notebook
from functions import find_not_responding_events,get_project_results,load_project_data,not_responding_logic
from testing import test_project_data
from runtime_functions import get_projects, find_projects_with_faulty_sensors
#import common_db as cdb

In [ ]:
if (min(df_irr_span_short.sm_val) > probeMaxSM - 0.5) and
                ((max(df_irr_span_short.sm_diff) > sm_hourly_diff*logic_parameters.LOW_RESPONSE_FACTOR)):

- You call the 'Project' class
- you bring in the data: project.load_sm_project_data
- you apply transformers on the data (to fill missing values, for example): project.apply_transformers()
 - and then you group the TS by depths:
    project.group_data_to_depths()

In [4]:
SM_HOURLY_DIFF_FIRST_DEPTH = 0.5
SM_HOURLY_DIFF_SECOND_DEPTH = 0.25
MIN_IRR_AMOUNT = 1
MAX_RESPONDING_EVENTS = 1
LOW_RESPONSE_FACTOR = 0.5
IRR_SPAN_END_AFTER_X_HOURS = 4


#project_list = [852093, 871812, 851995, 852015]
summary_df = pd.DataFrame()
yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
failed_list = []

In [22]:
#project_results = find_not_responding_events(project_data)

query = f"""
    SELECT distinct(pm.project_id)
    FROM projects_metadata pm
    JOIN soil_sensors_metadata ssm
    ON pm.project_id = ssm.project_id
    JOIN project_irrigation_spans_v2 pis
    ON pis.project_id = pm.project_id
    WHERE time_zone like '%Los_Angeles'
    AND season = 2023
    AND type_id IN (90, 91, 92, 98, 117, 118, 124, 127, 135, 137)
    AND start_date >= CAST((CAST('{start_date}' AS timestamp)) AS date)
    AND active=true
    ORDER BY project_id
    """
sql_importer = SqlImporter(query=query, database=c.database_production, user=c.user_production, password=c.password_production,
                                host=c.host_production, port=c.port_production, verbose=True)

sql_importer.get_data()
project_list = sql_importer.data
project_list

Loaded table with 2587 lines from projects_metadata


,project_id
0,848545
1,848547
2,849306
3,849307
4,849370
...,...
2582,878567
2583,879153
2584,879190
2585,879195


In [85]:
summary_df = pd.DataFrame()
yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
failed_list = []

project_list = [852093, 871812, 858363]
for p_id in project_list:#['project_id']:#[:-300]:
    #if p_id != 848545:
    #    continue
    try:
        project_data = load_project_data(project_id=p_id, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=False)
        if (project_data.df_irrigation.empty):
                continue
        if project_data.valid_project:
            project_df = get_project_results(project_data)
            summary_df = pd.concat([summary_df,project_df],axis=0)
            #display(project_df, summary_df)
        else:
            failed_list.append(p_id)
    except Exception as e:
        print(p_id, e)
        failed_list.append(p_id)
        pass

display(summary_df)
#display(summary_df[(summary_df.not_responding_events_count / summary_df.irrigation_events) > 0.5])

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff_in_events,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone,territory
0,852093,947777,12530,Farm Unit 5,989,almond,38,"[16.0, 31.0]",4,2,0.41,"{16.0: {'mean': 34.90937500000001, 'std': 2.42...","{2023-05-15 12:30:00, 2023-05-18 08:50:00, 202...",closed,2023-04-01 02:08:05.525108,49,{2023-05-15 12:30:00: probe 31.0 Not respondin...,https://app.phytech.com/12530/62308/852093,America/Los_Angeles,california
0,858363,878000,10728,Ranch 4860,138,pistachio,8,"[16.0, 31.0]",4,2,3.68,"{16.0: {'mean': 18.901197916666664, 'std': 0.7...","{2023-05-16 01:52:00, 2023-05-15 07:32:00}",closed,2023-05-05 01:11:25.687285,15,{2023-05-15 07:32:00: probe 16.0 Not respondin...,https://app.phytech.com/10728/51753/858363,America/Los_Angeles,california


In [98]:
def calc_sensor_status(df):
    if (df['percent_not_responding'] > 0.9) & (df.max_SM_diff_in_events < 0.5):
        return('fault')
    else:
        return('OK')

summary_df['date'] = dt.date.today().strftime("%Y-%m-%d")
summary_df['percent_not_responding'] = summary_df.apply(lambda row: (row.not_responding_events_count / row.irrigation_events) if row.irrigation_events>0 else 0, axis=1)
summary_df['sensor_status'] = summary_df.apply(lambda row: calc_sensor_status(row),axis=1)
#summary_df['max_diff'] = summary_df.apply(max(d['max_diff'] for d in summary_df.SM_statistics.values()))
summary_df['max_diff'] = summary_df.SM_statistics.apply(lambda x: max(pd.DataFrame(x).T['max_diff']))
summary_df['max_hourly_diff'] = summary_df.SM_statistics.apply(lambda x: max(pd.DataFrame(x).T['max_hourly_diff']))
summary_df

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff_in_events,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone,territory,date,percent_not_responding,sensor_status,max_diff
0,852093,947777,12530,Farm Unit 5,989,almond,38,"[16.0, 31.0]",4,2,0.41,"{16.0: {'mean': 34.90937500000001, 'std': 2.42...","{2023-05-15 12:30:00, 2023-05-18 08:50:00, 202...",closed,2023-04-01 02:08:05.525108,49,{2023-05-15 12:30:00: probe 31.0 Not respondin...,https://app.phytech.com/12530/62308/852093,America/Los_Angeles,california,2023-05-20,0.5,OK,7.69
0,858363,878000,10728,Ranch 4860,138,pistachio,8,"[16.0, 31.0]",4,2,3.68,"{16.0: {'mean': 18.901197916666664, 'std': 0.7...","{2023-05-16 01:52:00, 2023-05-15 07:32:00}",closed,2023-05-05 01:11:25.687285,15,{2023-05-15 07:32:00: probe 16.0 Not respondin...,https://app.phytech.com/10728/51753/858363,America/Los_Angeles,california,2023-05-20,0.5,OK,3.92


In [99]:
#summary_df#.drop('percent_not responding',inplace=True,axis=1)
dict_ = summary_df.remarks.reset_index(drop=True)[0]
dict_
#max(d['max_diff'] for d in dict_.values())
#max(pd.DataFrame(dict_).T['max_diff'])

{'2023-05-15 12:30:00: probe 16.0 Not responding to irrigation',
 '2023-05-15 12:30:00: probe 31.0 Not responding to irrigation',
 '2023-05-18 08:50:00: probe 31.0 Not responding to irrigation',
 '2023-05-18 14:43:00: probe 16.0 Not responding to irrigation',
 '2023-05-18 14:43:00: probe 31.0 Not responding to irrigation'}

In [77]:
cols = summary_df.columns.drop('area_name')

for row in summary_df.iterrows():
    try:
        query = """INSERT INTO public.DI_SM_not_responding_daily_results ({columns}) 
                    VALUES 
                    (
                    '{date}'::DATE
                    ,{project_id}
                    ,{sensor_id}
                    ,{area_id}
                    ,{area_name}
                    ,{territory}
                    ,{company_name}
                    ,{crop_name}
                    ,{variety_id}
                    ,{probe_depths}
                    ,{irrigation_events}
                    ,{not_responding_events_count}
                    ,{SM_statistics}
                    ,{event_timestamp}
                    ,{support_status}
                    ,{support_updated_at}
                    ,{days_since_task_complete}
                    ,{remarks}
                    ,{link}
                    ,{timezone}
                    ,{max_SM_diff}
                    ,{percent_not_responding}
                    ,{sensor_status}
                    )
                    ON CONFLICT (date,project_id, sensor_id)
                    DO UPDATE SET ({columns}) = (
                                                '{date}'::DATE
                                                ,{project_id}
                                                ,{sensor_id}
                                                ,{area_id}
                                                ,{area_name}
                                                ,{territory}
                                                ,{company_name}
                                                ,{crop_name}
                                                ,{variety_id}
                                                ,{probe_depths}
                                                ,{irrigation_events}
                                                ,{not_responding_events_count}
                                                ,{SM_statistics}
                                                ,{event_timestamp}
                                                ,{support_status}
                                                ,{support_updated_at}
                                                ,{days_since_task_complete}
                                                ,{remarks}
                                                ,{link}
                                                ,{timezone}
                                                ,{max_SM_diff}
                                                ,{percent_not_responding}
                                                ,{sensor_status}
                    );""".format(
                            columns = ', '.join(cols),
                            date = row[1]['date'],
                            project_id = row[1]['project_id'],
                            sensor_id = row[1]['sensor_id'],
                            area_id = row[1]['area_id'],
                            area_name = "a",#row[1]['area_name'],
                            territory = row[1]['territory'],
                            company_name = row[1]['company_name'],
                            crop_name = row[1]['crop_name'],
                            variety_id = row[1]['variety_id'],
                            probe_depths = row[1]['probe_depths'],
                            irrigation_events = row[1]['irrigation_events'],
                            not_responding_events_count = row[1]['not_responding_events_count'],
                            SM_statistics = row[1]['SM_statistics'],
                            event_timestamp = row[1]['event_timestamp'],
                            support_status = row[1]['support_status'],
                            support_updated_at = row[1]['support_updated_at'],
                            days_since_task_complete = row[1]['days_since_task_complete'],
                            remarks = row[1]['remarks'],
                            link = row[1]['link'],
                            timezone = row[1]['timezone'],
                            max_SM_diff = row[1]['max_SM_diff'],
                            percent_not_responding = row[1]['percent_not_responding'],
                            sensor_status = row[1]['sensor_status']
                            )
        sql_importer.query = query
        print(query)
        sql_importer.execute_command()
    except Exception as e:
        print(e)

INSERT INTO public.DI_SM_not_responding_daily_results (project_id, sensor_id, area_id, company_name, crop_name, variety_id, probe_depths, irrigation_events, not_responding_events_count, max_SM_diff, SM_statistics, event_timestamp, support_status, support_updated_at, days_since_task_complete, remarks, link, timezone, territory, sensor_status, date, percent_not_responding) 
                    VALUES 
                    (
                    '2023-05-20'::DATE
                    ,852093
                    ,947777
                    ,12530
                    ,a
                    ,california
                    ,989
                    ,almond
                    ,38
                    ,[16.0, 31.0]
                    ,4
                    ,2
                    ,{16.0: {'mean': 34.93284210526317, 'std': 2.429727993641571, 'max': 37.66, 'min': 29.97}, 31.0: {'mean': 33.91026315789476, 'std': 1.1318830282501668, 'max': 35.86, 'min': 32.41}}
                    ,{Timestamp('2023-05

Error while fetching data from PostgreSQL syntax error at or near "["
LINE 13:                     ,[16.0, 31.0]
                              ^



In [49]:
sql_importer = SqlImporter(database = c.database_research, user = c.user_research, password = c.password_research,
                                host = c.host_research, port = c.port_research, verbose=True)
 "select"
#sql_importer.get_postgress_conn()
summary_df.to_sql('DI_SM_not_responding_daily_results', sql_importer.get_postgress_conn())

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': relation "sqlite_master" does not exist
LINE 1: SELECT name FROM sqlite_master WHERE type='table' AND name=?...
                         ^


In [15]:
#df = summary_df[summary_df.irrigation_events!=0].copy()
#summary_df.groupby('project_id').count()
#df[df.project_id==848545]
#display(df[(df.not_responding_events_count / df.irrigation_events) > 0.5])
summary_df.shape

NameError: name 'summary_df' is not defined

In [30]:
summary_df['percent_not responding'] = summary_df.apply(lambda row: (row.not_responding_events_count / row.irrigation_events) if row.irrigation_events>0 else 0, axis=1)
summary_df.to_csv(DATA_WD+'/summary_df3.csv')

In [41]:
proj_data = load_project_data(project_id=849883, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=False)

In [29]:
def find_projects_with_faulty_sensors(df):
    failed_sensors_list = []
    filtered_df = df[(df['percent_not responding'] > 0.9) & (df.max_SM_diff < 0.5)]
    failed_sensors_list = list(filtered_df.sensor_id.values)
    return(failed_sensors_list)

summary_df['percent_not responding'] = summary_df.apply(lambda row: (row.not_responding_events_count / row.irrigation_events) if row.irrigation_events>0 else 0, axis=1)
df_project['percent_not responding'] = df_project.apply(lambda row: (row.not_responding_events_count / row.irrigation_events) if row.irrigation_events>0 else 0, axis=1)
find_projects_with_faulty_sensors(summary_df)

[934741, 878000]

## Testing

In [18]:
df_project = test_project_data(852093, debug=True, sql_debug=False)
df_project
df_project['remarks'][0]
df_project['SM_statistics'][0]

https://app.phytech.com/12530/62308/852093


,project_id,amount,start_ts,end_ts,psi,start_date,start,end,system_type
0,852093,25.26792,1683937020000,1683986760000,10.877447,2023-05-12,2023-05-12 17:17:00,2023-05-13 07:06:00,drip
1,852093,9.63168,1684179000000,1684197960000,8.992023,2023-05-15,2023-05-15 12:30:00,2023-05-15 17:46:00,drip
2,852093,5.18160,1684425000000,1684435200000,10.152284,2023-05-18,2023-05-18 08:50:00,2023-05-18 11:40:00,drip
3,852093,5.39496,1684446180000,1684456800000,9.862219,2023-05-18,2023-05-18 14:43:00,2023-05-18 17:40:00,drip


irrigation event: project_id                  852093
amount                     25.2679
start_ts             1683937020000
end_ts               1683986760000
psi                        10.8774
start_date              2023-05-12
start          2023-05-12 17:17:00
end            2023-05-13 07:06:00
system_type                   drip
Name: 0, dtype: object
2023-05-12 17:17:00

probe_depth 16.0
            local_time  sm_val  sm_diff        date
0  2023-05-12 16:00:00   32.25    -0.23  2023-05-12
1  2023-05-12 17:00:00   32.01    -0.24  2023-05-12
2  2023-05-12 18:00:00   37.05     5.04  2023-05-12
3  2023-05-12 19:00:00   37.25     0.20  2023-05-12
4  2023-05-12 20:00:00   37.35     0.10  2023-05-12
5  2023-05-12 21:00:00   37.48     0.13  2023-05-12
6  2023-05-12 22:00:00   37.51     0.03  2023-05-12
7  2023-05-12 23:00:00   37.54     0.03  2023-05-12
8  2023-05-13 00:00:00   37.57     0.03  2023-05-13
9  2023-05-13 01:00:00   37.55    -0.02  2023-05-13
10 2023-05-13 02:00:00   37.60    

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone,territory
0,852093,947777,12530,Farm Unit 5,989,almond,38,"[16.0, 31.0]",4,2,0.41,"{16.0: {'mean': 34.93284210526317, 'std': 2.42...","{2023-05-15 12:30:00, 2023-05-18 08:50:00, 202...",closed,2023-04-01 02:08:05.525108,49,{2023-05-15 12:30:00: probe 31.0 Not respondin...,https://app.phytech.com/12530/62308/852093,America/Los_Angeles,california


{'2023-05-15 12:30:00: probe 16.0 Not responding to irrigation',
 '2023-05-15 12:30:00: probe 31.0 Not responding to irrigation',
 '2023-05-18 08:50:00: probe 31.0 Not responding to irrigation',
 '2023-05-18 14:43:00: probe 16.0 Not responding to irrigation',
 '2023-05-18 14:43:00: probe 31.0 Not responding to irrigation'}

{16.0: {'mean': 34.93284210526317,
  'std': 2.429727993641571,
  'max': 37.66,
  'min': 29.97},
 31.0: {'mean': 33.91026315789476,
  'std': 1.1318830282501668,
  'max': 35.86,
  'min': 32.41}}

In [3]:
def test_project_data(p_id, debug=False, sql_debug=False):
    yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
    try:
        project_data = load_project_data(project_id=p_id, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=sql_debug)
    except Exception as e:
        print(e)
        return
    print(project_data.app_link)
    if project_data.valid_project & debug:
        display(project_data.df_irrigation[project_data.df_irrigation.amount > logic_parameters.MIN_IRR_AMOUNT])

    if project_data.valid_project:
        projects_df = get_project_results(project_data, debug)
        return(projects_df)
    else:
        return('project_data.valid_project', project_data.valid_project)

In [73]:
def get_first_depth(SM_dict):
    first_depth = list(SM_dict.keys())[0]
    print(first_depth, SM_dict[first_depth]['min'])
d = washington['SM_statistics'].iloc[20]

get_first_depth(eval(d))

16.0 25.89


In [88]:
summary_df = pd.read_csv(DATA_WD+'/projects_df_2023_05_15 15-08-24.csv').iloc[:,1:]
washington = summary_df[summary_df.territory == 'washington']
washington#['SM_statistics'][3:4].values[0]
#eval(washington['SM_statistics'][3:4].values[0])#.apply(lambda x: get_first_depth(eval(x)) if x !=  else None)

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone,territory,percent_not responding
20,850367,1149114,12935,Section 12,8678,apple,21,"[16.0, 31.0]",1,0,12.76,"{16.0: {'mean': 33.87623036649215, 'std': 2.32...",set(),completed,2023-04-29 03:10:22.091162,16.0,set(),https://app.phytech.com/12935/46900/850367,America/Los_Angeles,washington,0.0
22,850457,1117648,8291,Cherry Ridge Farms,8181,cherry,22,"[16.0, 31.0]",5,1,0.88,"{16.0: {'mean': 18.1020320855615, 'std': 0.509...","{Timestamp('2023-05-12 09:46:00'), Timestamp('...",NaN,NaN,NaN,{'2023-05-08 07:09:00: probe 31.0 Not respondi...,https://app.phytech.com/8291/38806/850457,America/Los_Angeles,washington,0.2
23,850570,1117596,11137,Colt- Jeff,7975,apple,21,[23.0],1,0,2.23,"{23.0: {'mean': 34.682239583333335, 'std': 0.4...",set(),completed,2023-03-14 03:10:19.265450,62.0,set(),https://app.phytech.com/11137/55249/850570,America/Los_Angeles,washington,0.0
24,850573,1125822,11137,Colt- Jeff,7975,apple,21,"[10.0, 25.0]",1,0,2.15,"{10.0: {'mean': nan, 'std': nan, 'max': nan, '...",set(),new,2023-04-30 23:39:37.710479,15.0,{'missing SM data for 10.0'},https://app.phytech.com/11137/55250/850573,America/Los_Angeles,washington,0.0
25,850665,1140618,8800,CPC - Cardinal Crest,9353,apple,21,"[16.0, 31.0]",5,0,3.35,"{16.0: {'mean': 14.190847457627116, 'std': 1.2...",set(),completed,2023-05-07 03:10:16.980802,8.0,set(),https://app.phytech.com/8800/41424/850665,America/Los_Angeles,washington,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,876765,1148058,19215,WHITSTRAN,7975,apple,144,"[15.0, 30.0]",3,0,0.94,"{15.0: {'mean': 28.457135416666663, 'std': 0.9...",{Timestamp('2023-05-12 08:10:00')},completed,2023-04-19 00:16:13.908251,26.0,{'2023-05-12 08:10:00: probe 30.0 Low sensor r...,https://app.phytech.com/19215/84607/876765,America/Los_Angeles,washington,0.0
2084,876766,1148055,19215,WHITSTRAN,7975,apple,143,"[16.0, 31.0]",3,0,10.18,"{16.0: {'mean': 29.74677083333334, 'std': 3.57...",set(),NaN,NaN,NaN,set(),https://app.phytech.com/19215/84609/876766,America/Los_Angeles,washington,0.0
2085,876768,1147999,19215,WHITSTRAN,7975,apple,143,"[16.0, 31.0]",2,0,11.50,"{16.0: {'mean': nan, 'std': nan, 'max': nan, '...",set(),NaN,NaN,NaN,{'missing SM data for 16.0'},https://app.phytech.com/19215/84611/876768,America/Los_Angeles,washington,0.0
2090,876962,1120686,9467,Gap Ranch,19740,blueberry,19,"[15.0, 30.0]",0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://app.phytech.com/9467/85309/876962,America/Los_Angeles,washington,0.0


In [36]:
summary_df = pd.read_csv(DATA_WD+'/projects_df_2023_05_15 15-08-24.csv').iloc[:,1:]
summary_df.shape
summary_df[summary_df.territory != 'washington'].shape
summary_df = summary_df[summary_df.territory != 'washington']
temp = pd.read_csv(DATA_WD+'/projects_df_2023_05_13 04-07-10.csv').iloc[:,1:]
summary_df.merge(right=temp[['project_id', 'Result ', 'Notes ']],
                 on='project_id',how='left').to_csv(DATA_WD+'/QA_table.csv')

(2092, 21)

(1444, 21)

In [138]:
query = """
select "id" project_id, plot_id from projects 
    where plot_id in (select territoriable_id from territory_entities where territoriable_type = 'Plot' and territory_id = 6)
    and state = 'installed'
    and season = 2023
"""
sql_importer = SqlImporter(query=query, database=c.database_ruby_production, user=c.user_production,
                           password=c.password_production, host=c.host_production, port=c.port_production,
                           verbose=False)

sql_importer.get_data()
proj_list = sql_importer.data
#list(proj_list['plot_id'])

query = """select metadata.area_id, metadata.plot_id, plot_name,crop_name,metadata.project_id, relevant_from, depth_cm, inner_index, sensor_id, type_id, relevant_untill
                    FROM public.project_metadata_vw metadata join public.soil_sensors_metadata sensors on metadata.project_id =sensors.project_id
                    where relevant_untill >=current_date and timezone like 'America/Los_Angeles'
                    --and plot_id in ({})
                    and state ='installed' 
                    and type_id in (90, 91, 92, 98, 117, 118, 124, 127, 135, 137)
                    and season =2023
                    and inner_index <=1
                    """
sql_importer = SqlImporter(query=query, database=c.database_research, user=c.user_research,
                           password=c.password_research, host=c.host_research, port=c.port_research,
                           verbose=False)
sql_importer.get_data()
metadata = sql_importer.data
df = metadata.merge(proj_list['plot_id'], on='plot_id',how='inner')

query = """
SELECT distinct soil.sensor_id,"date", average,minimum, maximum,maximum-minimum as daily_sm_change,
        average-lag(average) over (partition by soil.sensor_id,depth_cm order by soil.sensor_id,depth_cm,date) as daily_avg_diff
        FROM public.sensor_calculations_v2 as soil join public.soil_sensors_metadata as md
        on md.sensor_id =soil.sensor_id  and md.inner_index =soil.inner_index
        --where soil.sensor_id in (select sensor_id from sensors)
        where date>=current_date - integer '7' and depth_cm<=45 and soil.inner_index=0
"""
sql_importer.query = query
sql_importer.get_data()
sm_df = sql_importer.data
sm_df = sm_df.merge(df, on='sensor_id', how='inner')
sm_df

,sensor_id,date,average,minimum,maximum,daily_sm_change,daily_avg_diff,area_id,plot_id,plot_name,crop_name,project_id,relevant_from,depth_cm,inner_index,type_id,relevant_untill
0,810538,2023-05-09,22.195000,22.00,22.40,0.40,0.000000,8240,48154,RS09,Apple,856898,2022-12-27,15.0,0,90,2023-12-31
1,810538,2023-05-09,22.195000,22.00,22.40,0.40,0.000000,8240,48154,RS09,Apple,856898,2022-12-27,15.0,0,90,2023-12-31
2,810538,2023-05-09,22.195000,22.00,22.40,0.40,NaN,8240,48154,RS09,Apple,856898,2022-12-27,15.0,0,90,2023-12-31
3,810538,2023-05-09,22.195000,22.00,22.40,0.40,NaN,8240,48154,RS09,Apple,856898,2022-12-27,15.0,0,90,2023-12-31
4,810538,2023-05-10,21.791667,21.54,22.03,0.49,-0.403333,8240,48154,RS09,Apple,856898,2022-12-27,15.0,0,90,2023-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60755,1172422,2023-05-14,NaN,NaN,NaN,NaN,NaN,8419,39495,BLK 1,Apple,859567,2023-05-11,16.0,0,127,2023-12-31
60756,1172422,2023-05-14,NaN,NaN,NaN,NaN,NaN,8419,39495,BLK 1,Apple,859567,2023-05-11,31.0,1,127,2023-12-31
60757,1172422,2023-05-14,NaN,NaN,NaN,NaN,NaN,8419,39495,BLK 1,Apple,859567,2023-05-11,31.0,1,127,2023-12-31
60758,1172422,2023-05-14,NaN,NaN,NaN,NaN,NaN,8419,39495,BLK 1,Apple,859567,2023-05-11,31.0,1,127,2023-12-31


In [146]:
sus_list = list(sm_df[sm_df.average < 5]['project_id'].unique())
results = sm_df[(sm_df.project_id.isin(sus_list))].groupby('project_id')[['area_id','plot_id','average']].min()
results.reset_index(drop=False,inplace=True)

def display_app_link(row):
    project_link = 'https://app.phytech.com/%d/%d/%d' %(row.area_id, row.plot_id, row.project_id)
    #return(HTML("""<a href="%s">%s</a>"""  %(project_link, project_link)))
    return(project_link)
results['link'] = results.apply(lambda row: display_app_link(row),axis=1)
results.to_csv(DATA_WD+'/results.csv')

In [ ]:
def get_project_results(project_data, debug=False):
    project_results = find_not_responding_events(project_data,debug)
    df_irr = project_data.df_irrigation[project_data.df_irrigation.amount > MIN_IRR_AMOUNT]

    projects_df = pd.DataFrame(columns=['project_id','sensor_id','probe_depths','irrigation_events',
                                      'not_responding_events_count','max_SM_diff', 'event_timestamp',
                                       'support_status', 'support_updated_at','days_since_task_complete',
                                        'remarks', 'link','timezone'])
    project_dict = {}

    remarks = set()
    
    project_dict.update({'project_id' : project_data.project_id,
                         'sensor_id' : project_data.sensor_id,
                         'probe_depths': project_results.get('probe_depths'),
                         'irrigation_events' : len(df_irr),
                         'not_responding_events_count' : project_results['events_count'], #max_val,
                         'link' : project_data.app_link,
                         'timezone' : project_data['timezone']})
    
    # No not_responding events found
    if not project_results['events_details']: # empty dict = No events
        project_dict.update({'max_SM_diff': None,
                             #'probe_depths': None,
                             'event_timestamp' : None,
                             'remarks' : None})
    else:
        # find probe with max not responding events
        for d in project_results.get('probe_depths'):
            #remarks.add(project_results.get('events_details')[d]['remarks'])
            project_dict.update({'max_SM_diff': max(d['probe SM diff'] for d in project_results['events_details'].values()),
                                 #'probe_depths': project_results.get('probe_depths'),
                                 'event_timestamp' : project_results.get('event_timestamp'),
                                 #'remarks' : remarks
                                'remarks': project_results['remarks']})
    # Find support status information
    if not project_data.sensor_support_status:
        sensor_status = {'status': None, 'updated_at': None, 'days_since_task_complete': None}
    else:
        sensor_status = project_data.sensor_support_status_dict[0]
        sensor_status['days_since_task_complete'] = (dt.datetime.today().date() -
                                    sensor_status['updated_at'].date()).days
    
    project_dict.update({'support_status' : sensor_status['status'],
                         'support_updated_at' : sensor_status['updated_at'],
                         'days_since_task_complete': sensor_status['days_since_task_complete']})
    
    
    projects_df = pd.concat([projects_df, pd.DataFrame.from_dict(project_dict,orient='index').T], ignore_index=True)
    #projects_df['days_since_task_complete'] =  (dt.datetime.today().date() - projects_df['support_updated_at'][0].date()).days
    cols = ['project_id', 'sensor_id', 'probe_depths', 'irrigation_events',
       'not_responding_events_count', 'max_SM_diff', 'event_timestamp',
       'support_status', 'support_updated_at','days_since_task_complete', 'remarks', 'link', 'timezone']
    projects_df = projects_df[cols]
    
    return(projects_df)

In [14]:
def find_not_responding_events_old(project_data, debug=False):
    not_responding_SM_sensors_project_dict,probe_events_dict = {},{}
    event_timestamp = set()
    probe_depths = project_data.df_sm_data.depth_cm.unique()
    for probe_depth in probe_depths:
        probe_events_list = []
        
        if debug:
            print("\nprobe_depth", probe_depth)
            
        probe_dict = {}
        counter = 0
        probe_depth_index = list(probe_depths).index(probe_depth)
        
        for irr_event_counter,row in project_data.df_irrigation.iterrows():
            #probe_depth = project_data.df_sm_data.depth_cm.unique()[0]
            probe_local_saturation = project_data.local_saturation_by_depth[probe_depth_index][1]
            df = project_data.multi_depths_sm[probe_depth].reset_index(drop=False)
            df['date'] = df.local_time.dt.date

            if row['amount'] > MIN_IRR_AMOUNT:
                #################################
                # Not responding conditions:
                #
                # time frame irrigation span -1hr/ +3hr
                # soil moisture hourly diff > SM_HOURLY_DIFF according to depth
                # initial probe moisture is less than local saturation minus 0.5%
                #################################
                df_irr_span = df[(df.local_time > project_data.df_irrigation.start[irr_event_counter] - pd.Timedelta(hours=1)) 
                                 & (df.local_time < project_data.df_irrigation.end[irr_event_counter] + pd.Timedelta(hours=4))
                                ]
                if debug:
                    display(project_data.df_irrigation.start[irr_event_counter],project_data.df_irrigation.end[irr_event_counter],
                            df_irr_span)
                
                
                SM_HOURLY_DIFF = SM_HOURLY_DIFF_FIRST_DEPTH if probe_depth_index==0 else SM_HOURLY_DIFF_SECOND_DEPTH
                probe_dict['remarks'] = ''
                # check if probe responding = at least one hourly diff above SM_HOURLY_DIFF
                if max(df_irr_span.sm_diff) < SM_HOURLY_DIFF:
                    # check if the probe initial moisture is near local saturation 
                    if ("not_responding" in project_data.flag) | (max(df_irr_span.sm_val.iloc[:3]) < probe_local_saturation - 0.5):
                        counter+=1 # Not responding to irrigation
                        not_responding_sensor_id = int(project_data.df_sm_data[project_data.df_sm_data['depth_cm']==probe_depth].loc[0,'sensor_id'])
                        probe_events_list.append((probe_depth,project_data.df_irrigation.start[irr_event_counter]))
                        event_timestamp.add(project_data.df_irrigation.start[irr_event_counter])
                        probe_dict['remarks'] = 'Not responding to irrigation'
                # Find sensors with low response (short peak in sensor graph)
                    if (max(df_irr_span.sm_diff) > SM_HOURLY_DIFF*LOW_RESPONSE_FACTOR) & (max(df_irr_span.sm_diff) < SM_HOURLY_DIFF):
                        probe_dict['remarks'] = 'Low sensor respnse'

                if debug:
                    display(f"max hourly diff: {max(df_irr_span.sm_diff)}",
                            f"probe local saturation: {probe_local_saturation}",
                            f"initial probe moisture: {max(df_irr_span.sm_val.iloc[:3])}")
                    
            probe_dict['irrigation_events'] = len(project_data.df_irrigation)
            probe_dict['not_responding_events_count'] = counter
            probe_dict['events dates'] = probe_events_list
            probe_events_dict[probe_depth] = probe_dict

    not_responding_SM_sensors_project_dict.update({'probe_depths':list(probe_depths),
                                                 'event_timestamp':event_timestamp,
                                                 'events_details':probe_events_dict})
    return(not_responding_SM_sensors_project_dict)



In [412]:
query = f"""
            SELECT serial_number as sensor_id, updated_at, status
            FROM work_order_line_items
            WHERE serial_number = '{project_data.sensor_id}'
            --AND status not in ('closed','completed')
            order by updated_at desc
            limit 1
        """
sql_importer = SqlImporter(query=query, database=c.database_ruby_production, user=c.user_production, password=c.password_production, host=c.host_production, port=c.port_production, verbose=True)

sql_importer.get_data()
sql_importer.data

Loaded table with 1 lines from work_order_line_items



,sensor_id,updated_at,status
0,934741,2023-04-27 08:52:33.235331,new


def load_project_data(project_id, min_date, max_date, min_depth=10, max_depth=91, debug=False):
    from logic_parameters import default_latitude, default_height,default_max_depth
    from project_class_data_extract import Project
    project = Project(
        project_id=project_id,
        min_depth=min_depth,
        max_depth=default_max_depth,  # set the depth range we're interested in
        min_date=min_date,
        max_date=max_date,
        debug=debug)

    project.load_project_metadata()
    project.get_sm_depths()

    if len(project.depths_found) == 0:
        project.valid_project = False
        return project

    project.load_sm_project_data(min_depth=project.depths_found[0], max_depth=project.depths_found[-1], )  # change the min/max_depth if you dont want to load all depths
    if not project.valid_project:
        return project
    #project.load_project_weather_data(future=14)  # load the weather date until max_date + 14 days.
    project.apply_transformers()
    project.group_data_to_depths()
    project.load_irrigation_spans()
    project.find_probe_local_saturation()

    project.meta_data = {'project_id': project.project_id, 'latitude': project.latitude if project.latitude else default_latitude,
                         'height': project.height if project.height else default_height, 'app_link': project.app_link}
    project.sensor_support_status_dict = project.get_sensor_support_status()

    return project

In [29]:
def transform_data(project_data, depth, max_datetime, max_history_days):
    from config.logic_parameters import max_hours_after_peak, rise_th, peak_th
    dt = DataTransformer(project_data['meta_data'], project_data['multi_depths_sm'], project_data['weather_data'], depth=depth, max_history_days=max_history_days,
                         max_datetime=max_datetime, rise_th=rise_th, peak_th=peak_th)

    dt.collect_historical_peak2rise_events()
    dt.calc_sm_diffs()
    dt.extract_data_from_p2r(max_hours_after_peak=max_hours_after_peak)
    dt.add_lower_depth_data()
    dt.add_weather_data()
    project_data.update({'model_data': dt.data})

    model_data = dict(meta_data=project_data['meta_data'], model_data=project_data['model_data'])
    run_data = {'depth': dt.depth, 'max_datetime': dt.max_datetime, 'max_history_days': dt.max_history_days}
    model_data.update(run_data)
    return model_data

In [247]:
def load_irrigation_spans(p_id):
        query = f"""
            SELECT project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt
            FROM project_irrigation_spans_v2
            WHERE project_id = {p_id} 
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
        """
        query=f"""
        SELECT pis.project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt,
            irrigation_system_type system_type
            FROM project_irrigation_spans_v2 pis
            JOIN  projects_hierachy ph
            ON pis.project_id = ph.project_id
            WHERE pis.project_id = ({p_id}) OR 
                    pis.project_id = (SELECT project_id from projects_hierachy
                    WHERE  main_project_const_id = (SELECT main_project_const_id FROM projects_hierachy WHERE project_id = {p_id})
                    AND season = 2023
                    AND project_id <> {p_id})
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
            ORDER BY start_date
        """

        sql_importer = SqlImporter(query=query, database=c.database_production, user=c.user_production, password=c.password_production,
                                host=c.host_production, port=c.port_production, verbose=True)

        sql_importer.get_data()
        print(sql_importer.query)
        df_irrigation = sql_importer.data
        display(df_irrigation)#.rename(columns={'start_lt': 'start', 'end_lt': 'end'}, inplace=True))

load_irrigation_spans(855929)

Loaded table with 21 lines from project_irrigation_spans_v2

        SELECT pis.project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt,
            irrigation_system_type system_type
            FROM project_irrigation_spans_v2 pis
            JOIN  projects_hierachy ph
            ON pis.project_id = ph.project_id
            WHERE pis.project_id = (855929) OR 
                    pis.project_id = (SELECT project_id from projects_hierachy
                    WHERE  main_project_const_id = (SELECT main_project_const_id FROM projects_hierachy WHERE project_id = 855929)
                    AND season = 2023
                    AND project_id <> 855929)
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
            ORDER B

,project_id,amount,start_ts,end_ts,psi,start_date,start_lt,end_lt,system_type
0,855929,1.140883,1680813300000,1680816300000,35.968093,2023-04-06,2023-04-06 13:35:00,2023-04-06 14:25:00,drip
1,855929,1.346242,1680877800000,1680881340000,37.708484,2023-04-07,2023-04-07 07:30:00,2023-04-07 08:29:00,drip
2,855929,1.597237,1681404600000,1681408800000,41.334300,2023-04-13,2023-04-13 09:50:00,2023-04-13 11:00:00,drip
3,855929,2.395855,1681848900000,1681855200000,43.219724,2023-04-18,2023-04-18 13:15:00,2023-04-18 15:00:00,drip
4,855929,1.140883,1682095200000,1682098200000,41.334300,2023-04-21,2023-04-21 09:40:00,2023-04-21 10:30:00,drip
5,855929,1.140883,1682114100000,1682117100000,41.914431,2023-04-21,2023-04-21 14:55:00,2023-04-21 15:45:00,drip
6,855929,2.281767,1682171400000,1682177400000,34.807832,2023-04-22,2023-04-22 06:50:00,2023-04-22 08:30:00,drip
7,855929,0.000000,1682524200000,1682524200000,40.754170,2023-04-26,2023-04-26 08:50:00,2023-04-26 08:50:00,drip
8,855929,1.369060,1682607000000,1682610600000,43.074692,2023-04-27,2023-04-27 07:50:00,2023-04-27 08:50:00,drip
9,855929,1.140883,1682977800000,1682980800000,35.678028,2023-05-01,2023-05-01 14:50:00,2023-05-01 15:40:00,drip


In [38]:
query = """
DROP TABLE IF EXISTS DI_SM_not_responding_daily_results;
CREATE TABLE DI_SM_not_responding_daily_results (
   date VARCHAR,
   project_id INT NOT NULL,
   sensor_id INT NOT NULL,
   area_id INT,
   area_name VARCHAR,
   territory VARCHAR,
   company_name VARCHAR,
   crop_name VARCHAR,
   variety_id SMALLINT,
   probe_depths INTEGER[],
   irrigation_events SMALLINT,
   not_responding_events_count SMALLINT,
   SM_statistics JSONB,
   event_timestamp timestamp[],
   support_status VARCHAR,
   support_updated_at timestamp,
   days_since_task_complete SMALLINT,
   remarks JSONB,
   link VARCHAR,
   timezone VARCHAR,
   max_SM_diff DECIMAL,
   percent_not_responding DECIMAL,
   sensor_status VARCHAR
   PRIMARY KEY (date,project_id,sensor_id)
);
"""
sql_importer.query = query
sql_importer.execute_command()

KeyboardInterrupt: 